# Análisis de tendencias

## Tendencias de temas por tiempo

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

StatementMeta(, 7ef47d01-eb38-49de-b821-d34126957f53, 3, Finished, Available, Finished, False)

In [2]:
# Unir todas las tablas Silver
df_all = spark.table("silver_articles") \
    .union(spark.table("silver_blogs")) \
    .union(spark.table("silver_reports"))

StatementMeta(, 7ef47d01-eb38-49de-b821-d34126957f53, 4, Finished, Available, Finished, False)

In [3]:
# Join con tabla topic para obtener category
df_with_topic = df_all.join(
    spark.table("topic").select("keyword_id", "category"),
    on="keyword_id",
    how="left"
)

StatementMeta(, 7ef47d01-eb38-49de-b821-d34126957f53, 5, Finished, Available, Finished, False)

In [4]:
# ANÁLISIS ANUAL POR TEMA
df_trends_yearly = df_with_topic.groupBy("year", "category") \
    .agg(
        count("*").alias("total_articles"),
        countDistinct("news_site").alias("unique_sources")
    ) \
    .orderBy("year", col("total_articles").desc())

print(f"\nTendencias anuales por tema:")
display(df_trends_yearly)

StatementMeta(, 7ef47d01-eb38-49de-b821-d34126957f53, 6, Finished, Available, Finished, False)


Tendencias anuales por tema:


SynapseWidget(Synapse.DataFrame, c4b910ac-ab05-41b5-9936-0790e869eb69)

In [5]:
# ANÁLISIS ANUAL POR TEMA (TEMA PRINCIPAL)

# Calcular agregado anual
df_yearly_agg = df_with_topic.groupBy("year", "category") \
    .agg(
        count("*").alias("total_articles"),
        countDistinct("news_site").alias("unique_sources")
    )

# Ranking: obtener el tema #1 de cada año
window_year = Window.partitionBy("year").orderBy(col("total_articles").desc())

df_top_topic_per_year = df_yearly_agg \
    .withColumn("rnk", rank().over(window_year)) \
    .filter(col("rnk") == 1) \
    .drop("rnk") # Limpiamos la columna de apoyo

print(f"\nTema dominante por año (Top 1):")
display(df_top_topic_per_year)

StatementMeta(, 7ef47d01-eb38-49de-b821-d34126957f53, 7, Finished, Available, Finished, False)


Tema dominante por año (Top 1):


SynapseWidget(Synapse.DataFrame, 2e99ecae-ecff-4399-b0ee-a7c12351f772)

## Análisis de fuentes de noticias más activas

In [6]:
# Contar artículos por fuente
df_news_sources = df_all.groupBy("news_site") \
    .agg(count("*").alias("article_count"))

# Ventana para ID (estable en el tiempo)
window_id = Window.orderBy(col("news_site"))

df_news_sources = df_news_sources.withColumn(
    "source_id",
    row_number().over(window_id)
)

# Ranking por volumen (opcional)
window_rank = Window.orderBy(col("article_count").desc())

df_news_sources = df_news_sources.withColumn(
    "rank",
    row_number().over(window_rank)
)

# Orden final
df_news_sources = df_news_sources.select(
    "source_id",
    "rank",
    "news_site",
    "article_count"
)

# Mostrar resultados
print("\nFuentes más activas:")
display(df_news_sources)

# Guardar
df_news_sources.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .format("delta") \
    .saveAsTable("silver_news_sources")

StatementMeta(, 7ef47d01-eb38-49de-b821-d34126957f53, 8, Finished, Available, Finished, False)


Fuentes más activas:


SynapseWidget(Synapse.DataFrame, a1d07cc8-e96a-4b2b-8c94-d11dc1ec43ba)

## Normalizando las tablas silver

#### Crear tablas gold con el campo source_id

In [7]:
def normalize_fact_table(fact_table_name: str, dimension_table_name: str = "silver_news_sources"):
    """
    Reemplaza el campo news_site por id_source en una tabla de hechos y la sobrescribe.
    """
    # Cargar tabla de hechos
    df_fact = spark.table(f"silver_{fact_table_name}")
    
    # Cargar dimensión
    df_dim = spark.table(dimension_table_name).select("source_id", "news_site")
    
    # JOIN y reemplazo
    df_fact_norm = df_fact.join(
        df_dim,
        on="news_site",
        how="left"
    ).drop("news_site")
    
    # Sobrescribir tabla de hechos normalizada
    df_fact_norm.write \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .format("delta") \
        .saveAsTable(f"gold_{fact_table_name}")
    

StatementMeta(, 7ef47d01-eb38-49de-b821-d34126957f53, 9, Finished, Available, Finished, False)

In [8]:
normalize_fact_table("blogs")
normalize_fact_table("articles")
normalize_fact_table("reports")

StatementMeta(, 7ef47d01-eb38-49de-b821-d34126957f53, 10, Finished, Available, Finished, False)